In [1]:
import re
def clear_string(line, strip_symbol=None, replace_symbol=None):
    """
    :param line: a string
    :param strip_symbol:
    :param replace_symbol: a list of special symbol, need replace.
    :return:
    """
    if strip_symbol is not None:
        for sym in strip_symbol:
            line = line.strip(sym)

    if replace_symbol is not None:
        for sym in replace_symbol:
            line = line.replace(sym, "")

    return line
def split_string(line, split_symbol):
    """
    :param line: a string need be split
    :param split_symbol: a string: split symbol
    :return:
    """
    return list(filter(None, line.split(split_symbol)))


# access file to get sent(label and un-label)
def get_sent(sent_path, encode='utf-8', label_type=0, strip_symbol=None, replace_symbol=None):
    """
    :param sent_path: a file path, store sent in each line.
    :param label_type: 0 denote without label. 1 denote label first. 2 denote label second
    :param encode: a type of file encode
    :param replace_symbol: a list of special symbol need be replace
    :param strip_symbol: a list of symbol need be strip
    :return: a sent_col or sent_col and sent_label_col
    """
    assert os.path.exists(sent_path), "maybe send error file path"

    sent_col, label_col = [], []
    with open(sent_path, "r", encoding=encode, errors='ignore') as f:

        # traverse each line to parse sent
        for line in f.readlines():
            if line == "\n":
                continue

            # without label, only parse sentence.
            if label_type == 0:
                sent_col.append(clear_string(line, strip_symbol=strip_symbol, replace_symbol=replace_symbol))

            # label first, label + '\t' + sent.
            elif label_type == 1:
                clear_line = clear_string(line, strip_symbol=strip_symbol, replace_symbol=replace_symbol)
                cur_label, cur_sent = split_string(clear_line, split_symbol='\t')

                sent_col.append(cur_sent)
                label_col.append(int(cur_label))

            elif label_type == 2:
                clear_line = clear_string(line, strip_symbol=strip_symbol, replace_symbol=replace_symbol)
                cur_sent, cur_label = split_string(clear_line, split_symbol='\t')

                sent_col.append(cur_sent)
                label_col.append(int(cur_label))

    return sent_col if label_type == 0 else sent_col, label_col
def process_new_data(data):
    sent_col, sent_label_col, final_label_col = [], [], []
    label_col = []

    data = data.split('\n')
    for line in data:
        line = line.strip()

        # Check if the line is not empty
        if line:
            if line.startswith('{"subject":'):
                label_col.append(line)

            else:
                if label_col:
                    sent_col.append(line)
                    sent_label_col.append(1)  # Set the label to 1 if there's a dictionary
                    final_label_col.append(label_col)
                else:
                    sent_col.append(line)
                    sent_label_col.append(0)  # Set the label to 0 if no dictionary

                label_col = []

    # Add an empty dictionary for sentences that don't have a dictionary
    for i, label in enumerate(sent_label_col):
        if label == 0:
            final_label_col.insert(i, ["[[];[];[];[];[]]"])

    return sent_col, sent_label_col, final_label_col

# Example data
data = """
Title: Review iPhone 14: Thiết kế, hiệu năng, camera và pin	Title : Review iPhone 14 : Thiết kế , hiệu năng , camera và pin

iPhone 14 được ra mắt vào 09/2022, được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước.	iPhone 14 được ra mắt vào 09 / 2022 , được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước .
{"subject": ["1&&iPhone", "2&&14"], "object": ["28&&thế", "29&&hệ", "30&&trước"], "aspect": ["20&&phần", "21&&cứng"], "predicate": ["16&&nâng", "17&&cấp", "18&&nhẹ"], "label": "COM+"}
{"subject": ["1&&iPhone", "2&&14"], "object": ["28&&thế", "29&&hệ", "30&&trước"], "aspect": ["24&&phần", "25&&mềm"], "predicate": ["16&&nâng", "17&&cấp", "18&&nhẹ"], "label": "COM+"}

Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm.	Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm .
{"subject": [], "object": ["14&&thế", "15&&hệ", "16&&tiền", "17&&nhiệm"], "aspect": ["2&&ngoại", "3&&hình"], "predicate": ["7&&rất", "8&&ít", "9&&sự", "10&&khác", "11&&biệt"], "label": "EQL"}

Có một vài ý kiến trái chiều cho rằng, phiên bản iPhone 14 này không có điểm gì nổi bật, thậm chí còn cho rằng không nên bỏ tiền ra nâng cấp hoặc mua.	Có một vài ý kiến trái chiều cho rằng , phiên bản iPhone 14 này không có điểm gì nổi bật , thậm chí còn cho rằng không nên bỏ tiền ra nâng cấp hoặc mua .

Thực tế khi mua hoặc nâng cấp một thiết bị di động bạn thường quan tâm đến những yếu tố nào?	Thực tế khi mua hoặc nâng cấp một thiết bị di động bạn thường quan tâm đến những yếu tố nào ?

Thiết kế, hiệu năng, camera và pin hay đơn giản hơn chỉ là thương hiệu?	Thiết kế , hiệu năng , camera và pin hay đơn giản hơn chỉ là thương hiệu ?

Mặc dù đây không được đánh giá là chiếc iPhone tốt nhất trong tầm giá nhưng lại là chiếc Smartphone cân bằng giữa nhiều yếu tố.	Mặc dù đây không được đánh giá là chiếc iPhone tốt nhất trong tầm giá nhưng lại là chiếc Smartphone cân bằng giữa nhiều yếu tố .

Hãy cùng Mobileworld review iPhone 14, qua bài viết bên dưới nhé!	Hãy cùng Mobileworld review iPhone 14 , qua bài viết bên dưới nhé !
"""

sent_col, sent_label_col, final_label_col = process_new_data(data)

# Print the results
for sent, label, final_label in zip(sent_col, sent_label_col, final_label_col):
    print("Sentence:", sent)
    print("Label:", label)
    print("Final Label:", final_label)
    print()

Sentence: Title: Review iPhone 14: Thiết kế, hiệu năng, camera và pin	Title : Review iPhone 14 : Thiết kế , hiệu năng , camera và pin
Label: 0
Final Label: ['[[];[];[];[];[]]']

Sentence: iPhone 14 được ra mắt vào 09/2022, được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước.	iPhone 14 được ra mắt vào 09 / 2022 , được đánh giá là chỉ nâng cấp nhẹ về phần cứng cũng như phần mềm so với thế hệ trước .
Label: 0
Final Label: ['[[];[];[];[];[]]']

Sentence: Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm.	Về ngoại hình thì cũng có rất ít sự khác biệt so với thế hệ tiền nhiệm .
Label: 1
Final Label: ['{"subject": ["1&&iPhone", "2&&14"], "object": ["28&&thế", "29&&hệ", "30&&trước"], "aspect": ["20&&phần", "21&&cứng"], "predicate": ["16&&nâng", "17&&cấp", "18&&nhẹ"], "label": "COM+"}', '{"subject": ["1&&iPhone", "2&&14"], "object": ["28&&thế", "29&&hệ", "30&&trước"], "aspect": ["24&&phần", "25&&mềm"], "predicate": ["16&&nâng", "17&&cấp", "1

In [16]:
import json
class LabelParser(object):
    def __init__(self, label_col, elem_col, intermittent=False):
        """
        :param label_col:
        :param elem_col: ["entity_1", "entity_2", "aspect", "result"]
        :param intermittent: True denote "result" using intermittent representation
        """
        self.label_col = label_col
        self.elem_col = elem_col
        self.intermittent = intermittent

    def parse_sequence_label(self, split_symbol="&&", sent_col=None, file_type="vn"):
        """
        :param split_symbol:
        :param sent_col:
        :param file_type
        :return:
        """
        null_label = "[[];[];[];[];[]]"
        tuple_pair_col, elem_representation_col = [], []

        for index in range(len(self.label_col)):
            # For non-comparative sentences' label.
            if self.label_col[index][0] == null_label:
                tuple_pair_col.append([[(-1, -1)] * 5])
                elem_representation_col.append(self.init_label_representation())

            else:
                global_elem_col = self.init_label_representation()

                sequence_tuple_pair = []
                for pair_index in range(len(self.label_col[index])):
                    global_elem_col, cur_tuple_pair = self.parse_each_pair_label(
                        self.label_col[index][pair_index], global_elem_col, split_symbol, sent_col[index], file_type
                    )
                    sequence_tuple_pair.append(cur_tuple_pair)

                tuple_pair_col.append(sequence_tuple_pair)
                elem_representation_col.append(global_elem_col)

        return elem_representation_col, tuple_pair_col

    def parse_each_pair_label(self, sequence_label, global_elem_col, split_symbol, sent=None, file_type="vn"):
        """
        :param sequence_label:
        :param global_elem_col:
        :param split_symbol:
        :param sent:
        :param file_type:
        :return:
        """
        elem_representation = json.loads(sequence_label)
        
        print("as")
        tuple_pair_representation, result_elem = [], []

        for elem_index, each_elem in enumerate(elem_representation.values()):
            if elem_index == 3 and each_elem == "[]":
                print(elem_representation)

            elem_tuple = ()
            # not polarity
            if elem_index != len(elem_representation) - 1:
                number_char_col = each_elem
                if number_char_col != []:
                    if file_type == "cn":
                        s_index = int(split_string(number_char_col[0], split_symbol)[0])
                        e_index = int(split_string(number_char_col[-1], split_symbol)[0]) + 1
                    else:
                        s_index = int(split_string(number_char_col[0], split_symbol)[0]) - 1
                        e_index = int(split_string(number_char_col[-1], split_symbol)[0])
                    elem_tuple += (s_index, e_index)

                    if self.elem_col[elem_index] == "result":
                        result_elem += [s_index, e_index]

                    # [check sentence and label position]
                    # if sent is not None:
                    #     cur_elem_str = self.get_sub_elem(number_char_col, split_symbol)
                    #
                    #     if cur_elem_str != sent[s_index: e_index]:
                    #         print("----------------------------")
                    #         print(cur_elem_str)
                    #         print(sent[s_index: e_index])
                    #         print(s_index, e_index)
                    #         print(number_char_col)
                    #         print("----------------------------")

            else:
                polarity = each_elem
                elem_tuple += (polarity, polarity)
                print(elem_tuple)

                # 针对英文中可能存在空的情况
                if len(result_elem) == 0:
                    result_elem = [-1, -1]

                result_elem.append(polarity)

            elem_tuple = (-1, -1) if len(elem_tuple) == 0 else elem_tuple
            tuple_pair_representation.append(elem_tuple)

            if elem_index < 3 and elem_tuple != (-1, -1):
                global_elem_col[self.elem_col[elem_index]].add(elem_tuple)

        global_elem_col["result"].add(tuple(result_elem))

        return global_elem_col, tuple_pair_representation

    @staticmethod
    def get_sub_elem(number_char_col, split_symbol):
        """
        :param number_char_col:
        :param split_symbol:
        :return:
        """
        elem_str = ""
        for num_char in number_char_col:
            elem_str += split_string(num_char, split_symbol)[1]

        return elem_str

    def init_label_representation(self):
        return {key: set() for key in self.elem_col}
    
sent_col, sent_label_col, label_col = process_new_data(data)
LP = LabelParser(label_col, ["entity_1", "entity_2", "aspect", "result"])
label_col, tuple_pair_col = LP.parse_sequence_label("&", sent_col)
print(tuple_pair_col)

as
('COM+', 'COM+')
as
('COM+', 'COM+')
as
('EQL', 'EQL')
[[[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]], [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]], [[(0, 2), (27, 30), (19, 21), (15, 18), ('COM+', 'COM+')], [(0, 2), (27, 30), (23, 25), (15, 18), ('COM+', 'COM+')]], [[(-1, -1), (13, 17), (1, 3), (6, 11), ('EQL', 'EQL')]], [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]], [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]], [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]], [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]]]
